**Tabla de contenido**

- [Technical requirements](#Technical-requirements)
- [Comprendiendo el problema de la previsión](#Comprendiendo-el-problema-de-la-prevision)
- [Diseñando nuestro servicio de pronósticos](#Disenando-nuestro-servicio-de-pronosticos)
- [Seleccionando las herramientas](#Seleccionando-las-herramientas)
- [Training at scale](#Training-at-scale)
- [Sirviendo los modelos con FastAPI](#Sirviendo-los-modelos-con-FastAPI)
- [Esquemas de respuesta y solicitud](#Esquemas-de-respuesta-y-solicitud)

Este capítulo tratará de reunir algunos de los conocimientos que hemos adquirido en el libro hasta ahora con un ejemplo realista. Esto se basará en uno de los escenarios presentados en el Capítulo 1, Introducción a la Ingeniería de ML, donde se nos pedía construir un servicio de pronóstico para las ventas de artículos en la tienda. Discutiremos el escenario con cierto detalle y esbozaremos las decisiones clave que deben tomarse para que una solución sea una realidad, antes de mostrar cómo podemos emplear los procesos, herramientas y técnicas que hemos aprendido a lo largo de este libro para resolver partes clave del problema desde la perspectiva de la ingeniería de ML. Al final de este capítulo, deberías tener una visión clara de cómo construir tus propios microservicios de ML para resolver una variedad de problemas empresariales.

En este capítulo, cubriremos los siguientes temas:
- Comprender el problema de pronóstico
- Diseñar nuestro servicio de pronóstico
- Seleccionar las herramientas
- Capacitación a gran escala
- Servir los modelos con FastAPI
- Contenerizar y desplegar en Kubernetes

Cada tema nos brindará la oportunidad de recorrer las diferentes decisiones que debemos tomar como ingenieros trabajando en una entrega compleja de ML. ¡Esto nos proporcionará una referencia útil cuando salgamos y hagamos esto en el mundo real!

Con eso, ¡comencemos y construyamos un microservicio de previsión!

# Technical requirements

Los ejemplos de código en este capítulo serán más fáciles de seguir si tienes lo siguiente instalado y funcionando en tu máquina:

- Postman u otra herramienta de desarrollo de API
- Un gestor de clúster local de Kubernetes como minikube o kind
- La herramienta de línea de comandos de Kubernetes, kubectl

Hay varios archivos .yml de diferentes entornos conda dentro de la carpeta Chapter08 en el repositorio de GitHub del libro para los ejemplos técnicos, ya que hay algunos subcomponentes diferentes. Estos son:

- mlewp-capítulo08-entrenar: Esto especifica el entorno para ejecutar los scripts de entrenamiento.
- mlewp-capítulo08-servir: Esto especifica el entorno para la construcción del servicio web local FastAPI.
- mlewp-capítulo08-registrar: Esto proporciona la especificación del entorno para ejecutar el servidor de seguimiento MLflow.

En cada caso, crea el entorno Conda, como de costumbre, con:

- conda env create –f <ENVIRONMENT_NAME>.yml

Los ejemplos de Kubernetes en este capítulo también requieren cierta configuración del clúster y de los servicios que vamos a desplegar; estos se encuentran en la carpeta Chapter08/forecast bajo diferentes archivos .yml. Si estás usando kind, puedes crear un clúster con una configuración sencilla ejecutando:

- kind create cluster

O puedes usar uno de los archivos de configuración .yaml proporcionados en el repositorio:

- kind create cluster --config cluster-config-ch08.yaml

Minikube no ofrece una opción para leer una configuración de clúster .yaml como kind, por lo que, en su lugar, simplemente debes ejecutar:

- minikube start

para desplegar su clúster local.

# Comprendiendo el problema de la prevision

En el Capítulo 1, Introducción a la Ingeniería de ML, consideramos el ejemplo de un equipo de ML que ha sido encargado de proporcionar pronósticos de artículos a nivel de tiendas individuales en un negocio minorista. Los usuarios comerciales ficticios tenían los siguientes requisitos:

- Los pronósticos deberían mostrarse y ser accesibles a través de un panel basado en la web.
- El usuario debería poder solicitar pronósticos actualizados si es necesario.
- Los pronósticos deberían realizarse a nivel de tiendas individuales.
- Los usuarios estarán interesados en sus propias regiones/tiendas en cualquier sesión y no se preocuparán por las tendencias globales.
- El número de solicitudes de pronósticos actualizados en cualquier sesión será pequeño.

Dadas estas necesidades, podemos trabajar con el negocio para crear las siguientes historias de usuario, las cuales podemos ingresar en una herramienta como Jira, tal como se explica en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático. Algunos ejemplos de historias de usuario que cubren estas necesidades serían los siguientes:

- `Historia de Usuario 1`: Como planificador logístico local, quiero iniciar sesión en un panel por la mañana a las 09:00 y poder ver las previsiones de demanda de artículos a nivel de tienda para los próximos días, para poder entender la demanda de transporte con antelación.

- `Historia de usuario 2`: Como planificador logístico local, quiero poder solicitar una actualización de mi pronóstico si veo que está desactualizado. Quiero que el nuevo pronóstico se devuelva en menos de 5 minutos para poder tomar decisiones sobre la demanda de transporte de manera efectiva.

- `Historia de Usuario 3:` Como planificador logístico local, quiero poder filtrar las previsiones para tiendas específicas para poder entender qué tiendas están generando demanda y utilizar esto en la toma de decisiones.

Estas historias de usuario son muy importantes para el desarrollo de la solución en su conjunto. Como estamos enfocados en los aspectos de ingeniería de aprendizaje automático del problema, ahora podemos profundizar en lo que esto significa para la construcción de la solución.

Por ejemplo, el deseo de poder ver pronósticos de la demanda de artículos a nivel de tienda puede traducirse bastante bien en algunos requisitos técnicos para la parte de ML de la solución. Esto nos indica que la variable objetivo será el número de artículos requeridos en un día particular. Nos dice que nuestro modelo o modelos de ML deben ser capaces de trabajar a nivel de tienda, por lo que o tenemos un modelo por tienda o el concepto de la tienda puede incorporarse como algún tipo de característica.

De manera similar, el requisito de que el usuario quiera poder solicitar una actualización de mi pronóstico si veo que está desactualizado ... Quiero que el nuevo pronóstico se obtenga en menos de cinco minutos, lo que impone un claro requisito de latencia en el entrenamiento. No podemos construir algo que tarde días en reentrenarse, por lo que esto puede sugerir que un modelo único construido a partir de todos los datos puede no ser la mejor solución.

Finalmente, la solicitud de poder filtrar pronósticos para tiendas específicas nuevamente respalda la idea de que lo que construyamos debe utilizar algún tipo de identificador de tienda en los datos, pero no necesariamente como una característica para el algoritmo. Por lo tanto, podríamos querer empezar a pensar en una lógica de aplicación que tome una solicitud del pronóstico para una tienda específica, identificada por este ID de tienda, y luego el modelo de ML y el pronóstico se recuperen solo para esa tienda mediante algún tipo de búsqueda o recuperación que use este ID como filtro.

Al recorrer este proceso, podemos ver cómo solo unas pocas líneas de requisitos nos han permitido comenzar a desarrollar cómo abordaremos el problema en la práctica. Algunas de estas ideas y otras podrían consolidarse tras un poco de lluvia de ideas entre nuestro equipo para el proyecto en una tabla como la de la Tabla 8.1:

| Historia de Usuario | Detalles | Requisitos Técnicos |
|----------------------|-----------|---------------------|
| **1** | Como planificador logístico local, quiero iniciar sesión en un panel a las 09:00 de la mañana y poder ver los pronósticos de demanda de artículos a nivel de tienda para los próximos días, de modo que pueda comprender la demanda de transporte con anticipación. | • Variable objetivo = demanda de artículos. <br> • Horizonte de pronóstico = 1–7 días. <br> • Acceso por API para un panel o solución de visualización. |
| **2** | Como planificador logístico local, quiero poder solicitar una actualización de mi pronóstico si veo que está desactualizado. Quiero que el nuevo pronóstico se genere en menos de 5 minutos para poder tomar decisiones sobre la demanda de transporte de manera efectiva. | • Reentrenamiento ligero. <br> • Modelo por tienda. |
| **3** | Como planificador logístico local, quiero poder filtrar los pronósticos para tiendas específicas para entender qué tiendas están impulsando la demanda y usar esta información en la toma de decisiones. | • Modelo por tienda. |


# Disenando nuestro servicio de pronosticos

Los requisitos en la sección Comprender el problema de pronóstico son las definiciones de los objetivos que necesitamos alcanzar, pero no son el método para lograrlos. Aprovechando nuestra comprensión del diseño y la arquitectura del Capítulo 5, Patrones y Herramientas de Despliegue, podemos comenzar a desarrollar nuestro diseño.

Primero, debemos confirmar qué tipo de diseño debemos desarrollar. Dado que necesitamos solicitudes dinámicas, tiene sentido seguir la arquitectura de microservicios discutida en el Capítulo 5, Patrones y Herramientas de Despliegue. Esto nos permitirá construir un servicio que tenga como único objetivo recuperar el modelo adecuado de nuestro almacén de modelos y realizar la inferencia solicitada. Por lo tanto, el servicio de predicción debería tener interfaces disponibles entre el panel de control y el almacén de modelos.

Además, dado que un usuario puede querer trabajar con varias combinaciones de tiendas en una misma sesión y tal vez alternar entre las previsiones de estas, deberíamos proporcionar un mecanismo para hacerlo que sea eficiente.

desviaciones en este caso, sino por solicitudes dinámicas realizadas por el usuario. Esto añade un poco de complejidad, ya que significa que nuestra solución no debe reentrenarse por cada solicitud que llegue, sino ser capaz de determinar si vale la pena reentrenar para una solicitud concreta o si el modelo ya está actualizado. Por ejemplo, si cuatro usuarios inician sesión y están mirando la misma combinación de región/tienda/artículo y todos solicitan un reentrenamiento, está bastante claro que no necesitamos reentrenar nuestro modelo cuatro veces. En su lugar, lo que debería ocurrir es que el sistema de entrenamiento registre una solicitud, realice un reentrenamiento y luego ignore de manera segura las otras solicitudes.

Existen varias formas de servir modelos de ML, como hemos discutido varias veces a lo largo de este libro. Una forma muy poderosa y flexible es envolver los modelos, o la lógica de servicio del modelo, en un servicio independiente que se limite únicamente a realizar las tareas requeridas para el servicio de la inferencia de ML. Este es el patrón de servicio que consideraremos en este capítulo y es la clásica arquitectura de “microservicios”, donde diferentes piezas de funcionalidad se dividen en sus propios servicios distintos y separados.

Esto construye resiliencia y extensibilidad en tus sistemas de software, por lo que es un gran patrón con el que es recomendable familiarizarse. Esto también es particularmente adecuado para el desarrollo de sistemas de aprendizaje automático (ML), ya que estos deben consistir en servicios de entrenamiento, inferencia y monitoreo, como se describe en el Capítulo 3, De Modelo a Fábrica de Modelos. Este capítulo explicará cómo servir un modelo de ML utilizando una arquitectura de microservicios, empleando varios enfoques con diferentes ventajas y desventajas. Luego, podrás adaptar y desarrollar estos ejemplos en tus propios proyectos futuros.

Podemos reunir estos puntos de diseño en un diagrama de diseño de alto nivel, por ejemplo, en la Figura 8.1:


![Diseñoaplicacion](figures/diseno-aplicacion.png)

La siguiente sección se centrará en llevar estas consideraciones de diseño de alto nivel a un nivel de detalle más bajo mientras realizamos la selección de herramientas antes del desarrollo.




# Seleccionando las herramientas

Ahora que tenemos un diseño de alto nivel en mente y hemos anotado algunos requisitos técnicos claros, podemos comenzar a seleccionar el conjunto de herramientas que utilizaremos para implementar nuestra solución.

Una de las consideraciones más importantes en este aspecto será qué marco utilizamos para modelar nuestros datos y construir nuestra funcionalidad de pronóstico. Dado que el problema es un problema de modelado de series temporales con necesidad de reentrenamiento y predicción rápidos, podemos considerar los pros y los contras de algunas opciones que podrían ser adecuadas antes de continuar.

| Herramienta / Framework | Pros | Contras |
|--------------------------|------|---------|
| **Sklearn** | - Ya es entendida por casi todos los científicos de datos <br> - Sintaxis muy fácil de usar <br> - Mucho soporte de la comunidad <br> - Buen soporte para ingeniería de características y creación de *pipelines* | - No tiene capacidades nativas para series temporales <br> - Requiere más ingeniería de características para aplicar a modelos de series temporales <br> - Requiere más trabajo y entendimiento por parte del ingeniero/científico |
| **Prophet** | - Enfocado únicamente en pronósticos <br> - Tiene capacidades integradas de optimización de hiperparámetros <br> - Ofrece mucha funcionalidad lista para usar <br> - Suele dar resultados precisos en una gran variedad de problemas <br> - Proporciona intervalos de confianza directamente | - No es tan comúnmente usado como Sklearn (aunque sigue siendo relativamente popular) <br> - Los métodos subyacentes son sofisticados → puede llevar a un uso de *caja negra* <br> - No es escalable de forma inherente |
| **Spark MLlib** | - Escalable de manera nativa para grandes volúmenes de datos <br> - Buen soporte para ingeniería de características y *pipelines* | - No tiene capacidades nativas para series temporales <br> - Opciones de algoritmos relativamente limitadas |

Basado en la información de la Tabla 8.2, parece que la biblioteca Prophet sería una buena opción y ofrecería un buen equilibrio entre el poder predictivo, las capacidades deseadas de series temporales y la experiencia entre los desarrolladores y científicos del equipo.


Los científicos de datos podrían entonces usar esta información para construir una prueba de concepto, con un código muy similar al mostrado en el Capítulo 1, Introducción a la Ingeniería de ML, en la sección Ejemplo 2: Pronóstico con API, que aplica Prophet a un conjunto de datos minoristas estándar.

Esto cubre el paquete de ML que utilizaremos para la modelación, pero ¿qué hay de los otros componentes? Necesitamos construir algo que permita a la aplicación frontend solicitar que el backend realice acciones, por lo que es una buena idea considerar algún tipo de framework de aplicación web. También necesitamos considerar qué sucede cuando esta aplicación backend recibe muchas solicitudes, por lo que tiene sentido construirla pensando en la escalabilidad. Otra consideración es que tenemos la tarea de entrenar no uno, sino varios modelos en este caso de uso, uno para cada tienda minorista, y por lo tanto deberíamos intentar paralelizar el entrenamiento tanto como sea posible. Las últimas piezas del rompecabezas van a ser el uso de una herramienta de gestión de modelos y la necesidad de una capa de orquestación para desencadenar trabajos de entrenamiento y monitoreo de manera programada o dinámica.

Uniendo todo esto, podemos tomar algunas decisiones de diseño sobre las herramientas de bajo nivel necesarias además del uso de la biblioteca Prophet. Podemos resumirlas en la siguiente lista:

- `Prophet`: Conocimos la biblioteca de pronósticos Prophet en el Capítulo 1, Introducción a la Ingeniería de ML. Aquí proporcionaremos un análisis más profundo de esa biblioteca y cómo funciona antes de desarrollar una canalización de entrenamiento para crear los tipos de modelos de pronóstico que vimos para ese caso de uso minorista en el primer capítulo.

- `Kubernetes`: Como se discutió en el Capítulo 6, Escalando, esta es una plataforma para orquestar múltiples contenedores a través de clústeres de cómputo y permite construir soluciones de servicio de modelos de ML altamente escalables. Usaremos esto para alojar la aplicación principal.

- `Ray Train`: Ya conocimos a Ray en el Capítulo 6, Escalando. Aquí utilizaremos Ray Train para entrenar muchos modelos de pronóstico Prophet diferentes en paralelo, y también permitiremos que estos trabajos se activen mediante una solicitud al servicio web principal que maneja las solicitudes entrantes.

- `MLflow`: Conocimos MLflow en el Capítulo 3, De Modelo a Fábrica de Modelos, y se utilizará como nuestro registro de modelos.

- `FastAPI`: Para Python, los frameworks de backend web más utilizados suelen ser Django, Flask y FastAPI. Usaremos FastAPI para crear la aplicación principal de enrutamiento del backend que servirá las predicciones e interactuará con los demás componentes de la solución. FastAPI es un framework web diseñado para ser fácil de usar y para construir aplicaciones web de alto rendimiento, y actualmente está siendo utilizado por algunas organizaciones de alto perfil, incluyendo Uber, Microsoft y Netflix (según la página principal de FastAPI).

En este capítulo, nos vamos a centrar en los componentes de este sistema que son relevantes para servir los modelos a gran escala, ya que los aspectos de entrenamiento y reentrenamiento programados se cubrirán en el Capítulo 9, Construyendo un caso de uso de Extracción, Transformación y Aprendizaje Automático. Los componentes en los que nos centramos se pueden considerar como nuestra 'capa de servicio', aunque les mostraré cómo usar Ray para entrenar varios modelos de pronóstico en paralelo. Ahora que hemos hecho algunas elecciones de herramientas, ¡vamos a empezar a construir nuestro microservicio de ML!

# Training at scale

Cuando presentamos a Ray en el Capítulo 6, Escalando, mencionamos casos de uso donde los requisitos de datos o tiempo de procesamiento eran tales que tenía sentido utilizar un marco de computación paralela muy escalable. Lo que no se hizo explícito es que, a veces, estos requisitos provienen del hecho de que en realidad queremos entrenar muchos modelos, no solo un modelo con una gran cantidad de datos o un modelo más rápidamente. Esto es lo que haremos aquí.

El ejemplo de previsión minorista que describimos en el Capítulo 1, Introducción a la Ingeniería de ML, utiliza un conjunto de datos con varias tiendas minoristas diferentes. En lugar de crear un modelo que pudiera tener un número de tienda o un identificador como característica, una mejor estrategia podría ser entrenar un modelo de previsión para cada tienda individual. Esto probablemente dará una mayor precisión, ya que las características de los datos a nivel de tienda, que pueden aportar cierto poder predictivo, no se promediarán por el modelo al observar una combinación de todas las tiendas juntas. Por lo tanto, este es el enfoque que adoptaremos, y aquí es donde podemos usar el paralelismo de Ray para entrenar varios modelos de previsión simultáneamente.

Para utilizar Ray para hacer esto, necesitamos tomar el código de entrenamiento que teníamos en el Capítulo 1 y adaptarlo ligeramente. Primero, podemos reunir las funciones que teníamos para el preprocesamiento de los datos y para el entrenamiento de los modelos de previsión. Hacer esto significa que estamos creando un proceso serial que luego podemos distribuir para ejecutarlo en la porción de los datos correspondiente a cada tienda. Las funciones originales para el preprocesamiento y el entrenamiento de los modelos eran:

In [1]:
import ray
import ray.data
import pandas as pd
from prophet import Prophet

# función que prepara los datos para Prophet
def prep_store_data(df: pd.DataFrame,store_id: int = 4,store_open: int = 1) -> pd.DataFrame:
    df_store = df[
    (df['Store'] == store_id) &\
    (df['Open'] == store_open)
    ].reset_index(drop=True)
    df_store['Date'] = pd.to_datetime(df_store['Date'])
    df_store.rename(columns= {'Date': 'ds', 'Sales': 'y'}, inplace=True)
    return df_store.sort_values('ds', ascending=True)

# Función que entrena el modelo Prophet
def train_predict(df: pd.DataFrame,
                  train_fraction: float,
                  seasonality: dict) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, int]:
    train_index = int(train_fraction*df.shape[0])
    df_train = df.copy().iloc[0:train_index]
    df_test = df.copy().iloc[train_index:]

    model=Prophet(yearly_seasonality=seasonality['yearly'],
                  weekly_seasonality=seasonality['weekly'],
                  daily_seasonality=seasonality['daily'],
                  interval_width = 0.95)
    
    model.fit(df_train)
    predicted = model.predict(df_test)
    return predicted, df_train, df_test, train_index

Ahora podemos combinar todo esto en una sola función que tomará un DataFrame de pandas, preprocesará esos datos, entrenará un modelo de pronóstico Prophet y luego devolverá predicciones sobre el conjunto de prueba, el conjunto de entrenamiento, el conjunto de prueba y el tamaño del conjunto de entrenamiento, aquí etiquetado por el valor de train_index. Dado que deseamos distribuir la aplicación de esta función, necesitamos usar el decorador @ray.remote que presentamos en el Capítulo 6, Ampliando Escalamiento. Pasamos el argumento num_returns=4 al decorador para informar a Ray que esta función devolverá cuatro valores en una tupla.

In [2]:
@ray.remote(num_returns=4) #le dice a Ray que la función devolverá 4 objetos distintos.

def prep_train_predict(df: pd.DataFrame,
                       store_id: int,
                       store_open: int=1,
                       train_fraction: float=0.8,
                       seasonality: dict={'yearly': True, 'weekly': True, 'daily': False}) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, int]:
    df = prep_store_data(df, store_id=store_id, store_open=store_open)
    return train_predict(df, train_fraction, seasonality)

Ahora que tenemos nuestra función remota, solo necesitamos aplicarla. Primero, asumimos que el conjunto de datos se ha leído en un DataFrame de pandas de la misma manera que en el Capítulo 1, Introducción a la Ingeniería de ML. La suposición aquí es que el conjunto de datos es lo suficientemente pequeño para caber en la memoria y no requiere transformaciones computacionalmente intensivas. Esto tiene la ventaja de permitirnos usar la lógica de ingestión de datos relativamente inteligente de pandas, la cual permite varios formatos de la fila de encabezado, por ejemplo, así como aplicar cualquier lógica de filtrado o transformación que queramos antes de la distribución utilizando esa sintaxis de pandas ahora familiar. Si el conjunto de datos fuera más grande o las transformaciones más intensas, podríamos haber utilizado el método ray.data.read_csv() de la API de Ray para leer los datos como un Ray Dataset. Esto lee los datos en un formato de datos Arrow, que tiene su propia sintaxis de manipulación de datos.

In [3]:
import os
file_path = lambda file: os.path.join(os.getcwd(),'data/Rossmann Store Sales', file)
df = pd.read_csv(file_path('train.csv'))
df.head()

/tmp/ipykernel_56735/4262359401.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path('train.csv'))


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


Ahora, estamos listos para aplicar nuestro entrenamiento y prueba distribuidos. Primero, podemos recuperar todos los identificadores de las tiendas del conjunto de datos, ya que vamos a entrenar un modelo para cada una.

In [4]:
store_ids = df['Store'].unique()
print(store_ids)

[   1    2    3 ... 1113 1114 1115]


Antes de hacer cualquier otra cosa, inicializaremos el clúster de Ray usando el comando ray.init() que vimos en el Capítulo 6, Ampliación. Esto evita realizar la inicialización cuando llamamos por primera vez a la función remota, lo que significa que podemos obtener tiempos precisos del procesamiento real si realizamos alguna evaluación comparativa. Para mejorar el rendimiento, también podemos usar ray.put() para almacenar el DataFrame de pandas en el almacén de objetos de Ray. Esto evita que tengamos que replicar este conjunto de datos cada vez que ejecutamos una tarea. Al poner un objeto en el almacén, se devuelve un id, que luego puedes usar como argumento de función igual que el objeto original.

In [ ]:
ray.init(num_cpus=4) # inicializa el entorno de ejecución de Ray
df_id = ray.put(df) # lmacena un objeto en el “Object Store” de Ray

2025-10-07 14:27:01,772	INFO worker.py:1951 -- Started a local Ray instance.


(prep_train_predict pid=56956) 14:27:04 - cmdstanpy - INFO - Chain [1] start processing
(prep_train_predict pid=56956) 14:27:04 - cmdstanpy - INFO - Chain [1] done processing
(prep_train_predict pid=56956) 14:27:09 - cmdstanpy - INFO - Chain [1] start processing [repeated 141x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(prep_train_predict pid=56957) 14:27:09 - cmdstanpy - INFO - Chain [1] done processing [repeated 140x across cluster]
(prep_train_predict pid=56956) 14:27:14 - cmdstanpy - INFO - Chain [1] start processing [repeated 144x across cluster]
(prep_train_predict pid=56956) 14:27:14 - cmdstanpy - INFO - Chain [1] done processing [repeated 145x across cluster]
(prep_train_predict pid=56956) 14:27:19 - cmdstanpy - INFO - Chain [1] start processing [repeated 143x across cluster]
(prep_train_predict

Ahora, necesitamos enviar nuestras tareas de Ray al clúster. Cada vez que haces esto, se devuelve una referencia a un objeto de Ray que te permitirá recuperar los datos del proceso cuando usemos ray.get para recopilar los resultados. La sintaxis que he usado aquí puede parecer un poco complicada, pero podemos desglosarla pieza por pieza. La función principal de Python, map, simplemente aplica la operación de lista a todos los elementos del resultado de la sintaxis zip. El patrón zip(*iterable) nos permite descomprimir todos los elementos en la comprensión de listas, de manera que podamos tener una lista de referencias a objetos de predicción, referencias a datos de entrenamiento, referencias a datos de prueba y finalmente referencias a índices de entrenamiento. Observa el uso de df_id para referirse al dataframe almacenado en el almacén de objetos.

In [6]:
pred_obj_refs, train_obj_refs, test_obj_refs, train_index_obj_refs = map(
    list,zip(*([prep_train_predict.remote(df_id, store_id) for store_id in store_ids])),)

Luego necesitamos obtener los resultados reales de estas tareas, lo cual podemos hacer utilizando ray.get() como se discutió.

In [7]:
ray_results = {
'predictions': ray.get(pred_obj_refs),
'train_data': ray.get(train_obj_refs),
'test_data': ray.get(test_obj_refs),
'train_indices': ray.get(train_index_obj_refs)
}

Luego puedes acceder a los valores de estos para cada modelo con ray_results['predictions'][<index>] y así sucesivamente.

En el repositorio de Github, el archivo Chapter08/train/train_forecasters_ray.py ejecuta esta sintaxis y un ejemplo de bucle for para entrenar los modelos Prophet uno por uno de manera secuencial para comparación. Usando la biblioteca time para las mediciones y ejecutando el experimento en mi Macbook con cuatro CPUs siendo utilizadas por el clúster de Ray, pude entrenar 1,115 modelos Prophet en poco menos de 40 segundos usando Ray, en comparación con alrededor de 3 minutos y 50 segundos usando el código secuencial. ¡Eso es casi un aumento de velocidad de seis veces, sin hacer mucha optimización!

Ahora vamos a empezar a construir la capa de servicio para nuestra solución, de manera que podamos usar estos modelos de pronóstico para generar resultados para otros sistemas y usuarios.



# Sirviendo los modelos con FastAPI

El enfoque más simple y potencialmente más flexible para servir modelos de ML en un microservicio con Python es envolver la lógica de servicio dentro de una aplicación web ligera. Flask ha sido una opción popular entre los usuarios de Python durante muchos años, pero ahora el framework web FastAPI tiene muchas ventajas, lo que significa que debería considerarse seriamente como una mejor alternativa.

Algunas de las características de FastAPI que lo convierten en una excelente opción para un microservicio ligero son:

- `Validación de datos`: FastAPI utiliza y se basa en la biblioteca Pydantic, que permite hacer cumplir las indicaciones de tipo en tiempo de ejecución. Esto permite la implementación de pasos de validación de datos muy fáciles de crear que hacen que tu sistema sea mucho más robusto y ayudan a evitar comportamientos en casos extremos.

- `Flujos de trabajo asincrónicos integrados`: FastAPI te proporciona gestión de tareas asincrónicas desde el principio con las palabras clave async y await, por lo que puedes construir la lógica que necesitarás en muchos casos de manera relativamente fluida sin recurrir a bibliotecas adicionales.

- `Especificaciones abiertas`: FastAPI se basa en varios estándares de código abierto, incluyendo el estándar de API REST OpenAPI y el lenguaje declarativo JSON Schema, que ayuda a crear documentación automática de modelos de datos. Estas especificaciones ayudan a mantener el funcionamiento de FastAPI transparente y muy fácil de usar.

- `Generación automática de documentación`: El último punto mencionaba esto para los modelos de datos, pero FastAPI también genera automáticamente documentación para todo tu servicio utilizando SwaggerUI.

- `Rendimiento`: ¡Rápido está en el nombre! FastAPI utiliza el estándar Asynchronous Server Gateway Interface (ASGI), mientras que otros frameworks como Flask usan Web Server Gateway Interface (WSGI). El ASGI puede procesar más solicitudes por unidad de tiempo y lo hace de manera más eficiente, ya que puede ejecutar tareas sin esperar a que terminen las tareas anteriores. La interfaz WSGI ejecuta las tareas especificadas de manera secuencial y, por lo tanto, tarda más en procesar las solicitudes.

Entonces, las razones anteriores son por las que podría ser una buena idea usar FastAPI para servir los modelos de pronóstico en este ejemplo, pero ¿cómo vamos a hacer eso? Eso es lo que cubriremos ahora.

Cualquier microservicio debe recibir datos en algún formato específico; esto se llama la “solicitud”. Luego devolverá datos, conocidos como la “respuesta”. El trabajo del microservicio es procesar la solicitud, ejecutar una serie de tareas que la solicitud define o para las cuales proporciona entrada, crear la salida apropiada y luego transformarla en el formato de solicitud especificado. Esto puede parecer básico, pero es importante repasarlo y nos brinda el punto de partida para diseñar nuestro sistema. Está claro que tendremos que tener en cuenta los siguientes puntos en nuestro diseño:

- `Esquemas de solicitud y respuesta`: Dado que construiremos una API REST, es natural que especifiquemos el modelo de datos para las solicitudes y respuestas como objetos JSON con esquemas asociados. La clave al hacer esto es que los esquemas sean lo más simples posible y que contengan toda la información necesaria para que el cliente (el servicio que realiza la solicitud) y el servidor (el microservicio) puedan realizar las acciones apropiadas. Dado que estamos construyendo un servicio de pronóstico, el objeto de solicitud debe proporcionar suficiente información para permitir que el sistema genere un pronóstico adecuado, que la solución ascendente que llama al servicio pueda presentar a los usuarios o sobre el cual pueda realizar una lógica adicional. La respuesta deberá contener los puntos de datos del pronóstico reales o algún puntero hacia la ubicación del pronóstico.

- `Calcular`: La creación del objeto de respuesta, en este caso, una previsión, requiere cálculo, como se discute en el Capítulo 1, Introducción a la Ingeniería de ML. Una consideración clave al diseñar microservicios de ML es el tamaño de este recurso de cómputo y las herramientas adecuadas necesarias para ejecutarlo. Por ejemplo, si estás ejecutando un modelo de visión por computadora que requiere una GPU grande para realizar inferencias, no puedes hacerlo en el servidor que ejecuta el backend de la aplicación web si ese servidor es solo una máquina pequeña que utiliza CPU. De manera similar, si el paso de inferencia requiere la ingestión de un terabyte de datos, esto puede requerir el uso de un marco de paralelización como Spark o Ray ejecutándose en un clúster dedicado, que por definición tendrá que ejecutarse en máquinas diferentes de la aplicación web que sirve. Si los requisitos de cómputo son lo suficientemente pequeños y obtener datos de otra ubicación no es demasiado intenso, entonces podrías ejecutar la inferencia en la misma máquina que aloja la aplicación web.

- `Gestión de modelos`: Este es un servicio de ML, ¡así que, por supuesto, hay modelos involucrados! Esto significa, como se discutió en detalle en el Capítulo 3, De Modelo a Fábrica de Modelos, que necesitaremos implementar un proceso robusto para gestionar las versiones adecuadas de los modelos. Los requisitos de este ejemplo también implican que debemos ser capaces de utilizar muchos modelos diferentes de manera relativamente dinámica. Esto requerirá una consideración cuidadosa y el uso de una herramienta de gestión de modelos como MLflow, que también conocimos en el Capítulo 3. También debemos considerar nuestras estrategias para actualizar y revertir modelos; por ejemplo, ¿utilizaremos despliegues azul/verde o despliegues canario, como se discutió en el Capítulo 5, Patrones y Herramientas de Despliegue?

- `Monitoreo del rendimiento`: Para cualquier sistema de ML, como hemos discutido extensamente a lo largo del libro, el monitoreo del rendimiento de los modelos será críticamente importante, al igual que tomar las acciones apropiadas para actualizar o revertir estos modelos. Si los datos verdaderos para cualquier inferencia no pueden ser devueltos inmediatamente al servicio, entonces esto requerirá un proceso propio para reunir los datos verdaderos y las inferencias antes de realizar los cálculos deseados sobre ellos.

Estos son algunos de los puntos importantes que tendremos que considerar mientras construimos nuestra solución. En este capítulo, nos enfocaremos en los puntos 1 y 3, ya que el Capítulo 9 cubrirá cómo construir sistemas de entrenamiento y monitoreo que funcionen en un entorno por lotes. Ahora que sabemos algunas de las cosas que queremos tener en cuenta en nuestra solución, ¡vamos a empezar a construir!

# Esquemas de respuesta y solicitud

Si el cliente está solicitando una previsión para una tienda específica, como asumimos en los requisitos, entonces esto significa que la solicitud debería especificar algunas cosas. Primero, debería especificar la tienda, usando algún tipo de identificador de tienda que se mantenga en común entre los modelos de datos del microservicio de ML y la aplicación del cliente.

En segundo lugar, el rango de tiempo para la previsión debe proporcionarse en un formato apropiado que pueda ser fácilmente interpretado y utilizado por la aplicación. Los sistemas también deben contar con lógica para crear ventanas de tiempo de previsión adecuadas si no se proporcionan en la solicitud, ya que es perfectamente razonable asumir que si un cliente solicita “una previsión para la tienda X”, podemos asumir algún comportamiento predeterminado que proporcione una previsión para algún período de tiempo desde ahora hacia el futuro, lo cual probablemente será útil para la aplicación del cliente.

El esquema JSON de solicitud más simple que satisface esto es entonces algo como:

In [8]:
{
"storeId": "4",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
}

{'storeId': '4',
 'beginDate': '2023-03-01T00:00:00Z',
 'endDate': '2023-03-07T00:00:00Z'}

Dado que este es un objeto JSON, todos los campos son cadenas, pero están poblados con valores que serán fácilmente interpretables dentro de nuestra aplicación Python. La biblioteca Pydantic también nos ayudará a hacer cumplir la validación de datos, de lo cual hablaremos más adelante. Tenga en cuenta que también debemos permitir que la aplicación cliente solicite múltiples pronósticos, por lo que debemos permitir que este JSON se extienda para permitir listas de objetos de solicitud:

In [9]:
[
{
"storeId": "2",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
},
{
"storeId": "4",
"beginDate": "2023-03-01T00:00:00Z",
"endDate": "2023-03-07T00:00:00Z"
}
]

[{'storeId': '2',
  'beginDate': '2023-03-01T00:00:00Z',
  'endDate': '2023-03-07T00:00:00Z'},
 {'storeId': '4',
  'beginDate': '2023-03-01T00:00:00Z',
  'endDate': '2023-03-07T00:00:00Z'}]

Como se mencionó, nos gustaría construir la lógica de nuestra aplicación de manera que el sistema aún funcione incluso si el cliente solo realiza una solicitud especificando el store_id, y luego inferimos el horizonte de pronóstico apropiado desde ahora hasta algún momento en el futuro.

Esto significa que nuestra aplicación debería funcionar cuando se envía lo siguiente como el cuerpo JSON en la llamada a la API:


In [10]:
[
{
"storeId": "4",
}
]

[{'storeId': '4'}]

Para hacer cumplir estas restricciones en la solicitud, podemos usar la funcionalidad de Pydantic donde heredamos de la BaseModel de Pydantic y creamos una clase de datos que define los requisitos de tipo que acabamos de establecer:

In [11]:
from pydantic import BaseModel

class ForecastRequest(BaseModel):
    store_id: str
    begin_date: str | None = None
    end_date: str | None = None

Como puede ver, hemos hecho cumplir aquí que el store_id sea una cadena, pero hemos permitido que las fechas de inicio y fin del pronóstico se puedan dar como None. Si las fechas no se especifican, podríamos hacer una suposición razonable basada en nuestro conocimiento del negocio de que una ventana de tiempo útil para el pronóstico sería desde la fecha y hora de la solicitud hasta siete días después. Esto podría ser algo que se cambie o incluso se proporcione como una variable de configuración en la configuración de la aplicación. No trataremos ese aspecto en particular aquí para poder centrarnos en cosas más interesantes, por lo que esto se deja como un ejercicio divertido para el lector.

El modelo de pronóstico en nuestro caso estará basado en la biblioteca Prophet, como se discutió, y esto requiere un índice que contenga las fechas y horas para que se ejecute la previsión. Para generar esto basado en la solicitud, podemos escribir una función auxiliar sencilla:

In [12]:
import pandas as pd
import datetime

def create_forecast_index(begin_date: str = None, end_date: str = None):
    if begin_date == None:
        begin_date = datetime.datetime.now().replace(tzinfo=None)
    else:
        begin_date = datetime.datetime.strptime(begin_date,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)
    
    # Convert forecast end date
    if end_date == None:
        end_date = begin_date + datetime.timedelta(days=7)
    else:
        end_date = datetime.datetime.strptime(end_date,'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=None)

Esta lógica nos permite crear la entrada para el modelo de pronóstico una vez que se recupera de la capa de almacenamiento del modelo, en nuestro caso, MLflow.

El objeto de respuesta debe devolver el pronóstico en algún formato de datos, y siempre es imperativo que devuelvas suficiente información para que la aplicación cliente pueda asociar cómodamente el objeto devuelto con la respuesta que provocó su creación. Un esquema simple que cumpla con esto sería algo como:

In [ ]:
[
    {
        "request": {
        "store_id": "4",
        "begin_date": "2023-03-01T00:00:00Z",
        "end_date": "2023-03-07T00:00:00Z"
        },

    "forecast": [
        {

            "timestamp": "2023-03-01T00:00:00",
            "value": 20716
        },
        {
            "timestamp": "2023-03-02T00:00:00",
            "value": 20816
        },
        {
            "timestamp": "2023-03-03T00:00:00",
            "value": 21228
        },
        {
            "timestamp": "2023-03-04T00:00:00",
            "value": 21829
        },
        {
            "timestamp": "2023-03-05T00:00:00",
            "value": 21686
        },
        {
            "timestamp": "2023-03-06T00:00:00",
            "value": 22696
            },
{
"timestamp": "2023-03-07T00:00:00",
"value": 21138
}
            ]
    }
]